In [1]:
from hashlib import sha256
import random

class Digital_Signature:
    sender = "unknown"
    receiver = "unknown"
    
    def __init__(self):
        print("=== Digital Signature ===")
        self.sender = "Jack"
        self.receiver = "Mary"
        
        print("Sender: " + self.sender + " | Receiver: " + self.receiver, end="\n\n")
    
    # This method/function is just related with the interface
    def show_menu(self):
        # Clearing the Screen
        print("Operations: ", end = "\n")
        print("\tG - Generate Primes :: ", end = "\n")
        print("\tK - Generate Keys :: ", end = "\n")
        print("\tS - Sign a text :: ", end = "\n")
        print("\tV - Verify signature :: ", end = "\n")
        print("\tE - Exit :: ", end = "\n")
        
        print("Please, enter the operation: ", end = "\n")
        option = input()
        option = option.strip()
        
        print(end = "\n")
        
        if option == "G" or option == 'g':
            self.show_generate_primes()
        elif option == "K" or option == 'k':
            self.show_generate_keys()
        elif option == "S" or option == 's':
            self.show_sign_text()
        elif option == "V" or option == 'v':
            self.show_verify_signature()
        elif option == "E" or option == 'e':
            self.dismiss()
        else:
            print("Invalid option. Please, enter a sigle letter according to the desired operation!", end = "\n")
            self.show_menu()
    
    # A really big prime number is generated
    def generate_primes(self):
        upper_bound = 2**(256) - 1
        lower_bound = 2**(254) - 1
        
        p = random_prime(upper_bound, False, lower_bound)
        q = random_prime(upper_bound, False, lower_bound)
        while(p == q):
            q = random_prime(upper_bound, False, lower_bound)
        return p, q
        
    def show_generate_primes(self):
        p, q = self.generate_primes()
        
        print(f'First prime (p): {p}' , end="\n")
        print(f'Second prime (q): {q}', end="\n")
        print(end="\n")
        
        self.closeOrNot()    
        
    def show_generate_keys(self):
        print("Enter the first prime(p): ")
        p = Integer(input())
        print("Enter the second prime(q): ")
        q = Integer(input())
        
        public_key, private_key = self.rsa_key_generation(p, q)
        
        print(end="\n")
        print(end="\n")
        print(f'Public key: {public_key}', end = "\n")
        print(end="\n")
        print(f'Private key: {private_key}', end = "\n")
        
        print(end="\n")
        
        self.closeOrNot()
    
    def show_sign_text(self):
        print("Enter the text to be signed: ")
        text = input().strip()
        
        print("Enter the private key (p): ")
        e = Integer(input().strip())
        
        print("Enter the private key (n): ")
        n = Integer(input().strip())
        
        private_key = e, n
        
        signature = self.sign_text(text, private_key)
        
        print("Signature of the given text: " + str(signature), end = "\n")
        print(end="\n")
        
        self.closeOrNot()
        
    # Interact with the user, so that we can verify the inputs
    def show_verify_signature(self):
        print("Enter the text to be verified: ")
        text = input().strip()
        
        print("Enter the corresponding signature to check: ")
        signature = input().strip()
        
        print("Enter the public key (d) of the given pair: ")
        d = Integer(input().strip())
        
        print("Enter the public key (n) of the given pair: ")
        n = Integer(input().strip())
        
        public_key = d, n
        
        # SIDE NOTE: Here I used a trick to cause heartbeating in terms of expectations on the respose :)
        # 0% 10% ... 100%. The difference between the percentages is randomic :):):) ...

        print("\nCkecking ... ", end = '')
        
        load = int(0)
        while load <= 100:
            print(f'{load}%', end = ' ')
            rand = int(random.randint(0, 100))
            sleep(round((rand+1)/24))
            load += rand
        
        print("100%", end="\n\n")
        
        print("Verdict: ", end = '')
        
        # After the trick is done, the response is provided right away
        res = self.verify_signature(text, signature, public_key)
        if res == True:
            print("Success! The signature is valid.", end = "\n")
        else:
            print("Fail! Invalid signature.", end = "\n")
        
        print(end="\n")
        self.closeOrNot()
    
    def dismiss(self):
        # This one is just a complement for recognition
        print(end="\n")
        print("THANK YOU FOR USING MY SCRIPT :)")
        print("Copyright© Application of Discrete Models ELTE 2022, Alfredo Martins (Student) & Dr. Mohammed B. M. Kamel (Professor)")
        quit()
              
    def closeOrNot(self):
        print("Would you like to close the program? Enter 'Y' to close", end = "\n")
        option = input().strip() # Remove empty spaces in the left and right
              
        if option == "Y" or option == "y":
            self.dismiss()
        else:
            sleep(1) # Put a delay of 1 second
            self.show_menu() # Go back to the menu

    def rsa_key_generation(self, p, q):

        n = p*q
        phi = (p-1)*(q-1) #euler_phi(n) is slower

        e = ZZ.random_element(phi) # [2, n - 1] -> Good
        while gcd(phi, e) != 1 or e < 2:
            e = ZZ.random_element(phi)

       # By applying Bézout theorem, we get the coefficients as we learned
        bezout_coefficient = xgcd(e, phi)
        d = Integer(mod(bezout_coefficient[1], phi)) # Inverse_mod function <--- ???

        mod_result = mod(d*e, phi)
        if mod_result == 1: #We are good to go
            public_key = d, n # These two pairs forms the PUBLIC KEY
            private_key = e, n # These two pairs forms the PRIVATE KEY

            return public_key, private_key # Keys generated
        return None
    
    # This method is the so called Binary Exponentiation, used to find a^b mod m, in fast and efficienty way
    def bin_pow(self, a, b, m):
        a = mod(a, m) # a = a%m
        res = Integer(1) # Indentity element of multiplication is 1. Everybody knows that :)
        while b > 0:
            if b & 1: # If b is odd
                res = mod(res * a, m) # res = (res*a)%m
            a = mod(a * a, m)
            b >>= 1 # Shit bits to the right, basically, It is b = b/2
        return res


    def rsa_encryption(self, m, private_key):
        e, n = private_key
        c = self.bin_pow(m, e, n) # Formula used in the report
        return c
    
    # Used for debugging/testing purpose.
    def get_char_representation(self, text):
        list_text = []
        for c in text:
            list_text.append(ord(c))
                
        final_value = Integer()
        for x in list_text:
            final_value = final_value*100 + x
        
        return final_value

    def rsa_decryption(self, c, public_key):
        d, n = public_key
        
        c = Integer(mod(c, n)) # Normal integer does not support huge number, so, let's convert them
        n = Integer(n)
        d = Integer(d)
        #m = power_mod(c, d, n) # Using the normal/builtin function, there is a problem with % working with hash values
        m = self.bin_pow(c, d, n) # Formula used in the report
        return m
    
    # Before encript, let's add more security! Let's hash the value, as explained in the report
    def sign_text(self, plain_text, private):
        m = self.get_hash_text(plain_text) # Hash the value using SHA256
        signature = self.rsa_encryption(m, private) # Encript the digiest using RSA
        return signature
        
    def get_hash_text(self, text):      
        m = int.from_bytes(sha256(text.encode()).digest(), byteorder='big') # Get the Digest
        return m
        
    def verify_signature(self, plain_text, signature, public_key):
        text_hash = self.get_hash_text(plain_text) # Hash the value
        
        original_text_hash = self.rsa_decryption(signature, public_key) # Decrypt using RSA
        
        #print(f"Decrypted: {original_text_hash}", end = "\n")
        #print(f"Decrypted hash functon: {self.get_hash_text(str(original_text_hash))}", end = "\n")
        #print(f"Hashed text: {text_hash}", end = "\n")
        
        if original_text_hash == text_hash:
            return True
        return False

In [4]:
obj = Digital_Signature()
obj.show_menu()

=== Digital Signature ===
Sender: Jack | Receiver: Mary

Operations: 
	G - Generate Primes :: 
	K - Generate Keys :: 
	S - Sign a text :: 
	V - Verify signature :: 
	E - Exit :: 
Please, enter the operation: 
K

Enter the first prime(p): 
111
Enter the second prime(q): 
97


Public key: (10187, 10767)

Private key: (4643, 10767)

Would you like to close the program? Enter 'Y' to close
q
Operations: 
	G - Generate Primes :: 
	K - Generate Keys :: 
	S - Sign a text :: 
	V - Verify signature :: 
	E - Exit :: 
Please, enter the operation: 
S

Enter the text to be signed: 
Genilson.
Enter the private key (p): 
4643
Enter the private key (n): 
10767
Signature of the given text: 4513

Would you like to close the program? Enter 'Y' to close
q
Operations: 
	G - Generate Primes :: 
	K - Generate Keys :: 
	S - Sign a text :: 
	V - Verify signature :: 
	E - Exit :: 
Please, enter the operation: 
V

Enter the text to be verified: 
Genilson
Enter the corresponding signature to check: 
4513
Enter the

In [24]:
obj = Digital_Signature()
#obj.show_menu()
text = "HELLOWORLD"
res = obj.get_char_representation(text)
print(res)

=== Digital Signature ===
Sender: Jack | Receiver: Mary

72697676798779827668


In [ ]:
obj = Digital_Signature() m = 72697676798779827668 e = 1850567623300615966303954877 n = 4951760154835678088235319297

signature = obj.rsa_encryption(m, (e, n))

c = power_mod(m, e, n) print(c)

c = 630913632577520058415521090 d = 4460824882019967172592779313 n = 4951760154835678088235319297

res = obj.rsa_decryption(c, (d, n))

power_mod(c, d, n)